# US Commercial EN SE Champions Mashup Script
#### Created by Jeff Alexander (jefalexa@cisco.com)
#### Updated on 2/28/19 by Jeff Alexander (jefalexa@cisco.com)

#### Description
This script is used to maintain an accurate list of the designated EN Champions for reference and distribution.  It combines Directory Org structure, SAGE technical expertise and SEVT membership information downloaded by the user into a comprehensive and up to date list.  It then updates the published SmartSheet for distribution.  

#### SmartSheet Info
- Name:  "EN Champions"
- URL:  https://app.smartsheet.com/sheets/JFc989gxfXM7hF4vhP2f2QpQQ28XgwMq6Ch6xhJ1?view=grid
- SheetID:  "809447254714244"


## Prepare Environment
#### If not already there, add your SmartSheet API Key to the "api_keys.py" file
#### Ensure that this script is running in the provided Virtual Environment container to avoid module dependancy issues



### Pull directory dump for “Who is my Champion?” correlation
https://labtools.cisco.com/general/orgchart.php?tops=lydohert&photos=none

Click "Raw Data"

### SAGE data
https://wwss.cisco.com/#/site/WWSE/views/Sage-BasicDashboards_0/DetailedExpertiseDash/jefalexa@cisco.com/AreaENChampions?:iid=4

Click on the main table to select it, then "Download" >> "Download all rows as a text file"
        
### SEVT membership / attendance
https://wwss.cisco.com/#/site/WWSE/views/Sage-BasicDashboards_0/VTMembersDash/jefalexa@cisco.com/USCENSEVTMember?:iid=6

Click on the main table to select it, then "Download" >> "Download all rows as a text file"

Archive the old files and move the new files to the local working directory
- searchresult.csv = Directory Info
- Detailed_Expertise_data.csv = SAGE Data
- VT_Members_data.csv = SEVT Membership



## Run the Code
Once you have verified that the new files are copied locally, run the code by selecting "Cell" --> "Run All" from the top menu bar.  It will run each of the cells below in sequence, logging progress as it goes.  The cells are broken up into logical blocks to make troubleshooting and updates eaiser.  


## Import Modules and Data

In [ ]:
import string, re, sys, time, csv
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import datetime
import smartsheet
import logging
import os
import json
import api_keys

print("STARTING - {0}".format(datetime.datetime.now()))

# Define imput/output files
input_file1 = "Detailed_Expertise_data.csv"
input_file2 = "VT_Members_data.csv"
input_file3 = "searchresult.csv"
input_file4 = "dept_mapping.csv"
output_file = "default_output_file.xlsx"

# Import CSV files as Pandas Data Frames
INPUT_SAGE1 = pd.read_csv(input_file1, quotechar='"', low_memory=False, sep='\t', encoding='utf-16')
INPUT_SAGE2 = pd.read_csv(input_file2, quotechar='"', low_memory=False, sep='\t', encoding='utf-16')
INPUT_DIR1 = pd.read_csv(input_file3, quotechar='"', low_memory=False, encoding='utf-8-sig')
INPUT_DIR2 = pd.read_csv(input_file4, quotechar='"', low_memory=False, encoding='utf-8-sig')

print("Initialize SmartSheet Client...")
# Initialize client
ss = smartsheet.Smartsheet(api_keys.ss_access_token)
# Make sure we don't miss any error
ss.errors_as_exceptions(True)

# Log all calls
logging.basicConfig(filename='rwsheet.log', level=logging.INFO)
    

# Define Functions
print("Initialize Functions...")
def get_column_names(sheet_id):
    """Returns a dict of the column names and their corresponding ID's in the working Sheet (sheet_id)"""
    column_names = {}
    column_list = ss.Sheets.get_columns(sheet_id).to_dict()
    for i in column_list['data']:
        column_names[i['title']] = i['id']
    return(column_names)

def get_row_search(sheet_id, search_string, search_column):
    """Searches the working sheet (sheet_id) for a term (search_string), 
    then verifies that the term is in the specific colum you are referencing (search_column)
    returns a list of column ID's that match both"""
    sr = ss.Sheets.search_sheet(sheet_id, search_string).to_dict()['results']
    sr_list = []
    for x in sr:
        my_row_d = ss.Sheets.get_row(sheet_id, x['objectId']).to_dict()
        for y in my_row_d['cells']:
            if (y['columnId'] == column_names[search_column]):
                if (y['value'] == search_string):
                    sr_list.append(x['objectId'])
    return(sr_list)


def update_cell(sheet_id, column_id, row_id, new_value):
    """Updates the value of a give cell (column_id, row_id) with a new value (new_value)"""
    # Build new cell value
    new_cell = ss.models.Cell()
    new_cell.column_id = column_id
    new_cell.value = str(new_value)
    new_cell.strict = False

    # Build the row to update
    new_row = ss.models.Row()
    new_row.id = row_id
    new_row.cells.append(new_cell)

    # Update row
    updated_row = ss.Sheets.update_rows(sheet_id, [new_row])
    

def update_cells(sheet_id, row_search, column_search, column_name, new_value):
    """Gets a list of rows (row_list) by calling get_row_search with the search term (row_search) and column name to search in (column_search)
    gets the column_id from the column_name and updates that cell with the new_value in each of the rows matched.
    Returns the number of rows matched by the search and the number updated as a string.  """
    row_list = get_row_search(sheet_id, row_search, column_search)
    column_id = column_names[column_name]
    row_count = len(row_list)
    update_count = 0
    for r in row_list:
        update_cell(sheet_id, column_id, r, new_value)
        update_count += 1
    return("Updated {0} of {1} rows".format(update_count, row_count))

def update_cells_by_dict(update_dict):
    progress_total = len(update_dict)
    progress_count = 1 
    for y in update_dict:
        print("Processing {0} of {1} UID's".format(progress_count, progress_total))
        progress_count += 1 
        L1_id = y
        L1_value = update_dict[L1_id]
        for L2_id in L1_value:
            print(update_cells(sheet_id, L1_id, "uid", L2_id, L1_value[L2_id]))
   
def set_expertise(x):
    Exp_Found = re.compile(".*Foundational.*",re.IGNORECASE)
    Exp_Adv = re.compile(".*Advanced.*",re.IGNORECASE)
    Exp_ID = re.compile(".*In-Depth.*",re.IGNORECASE) 
    try:
        if Exp_Found.match(x):
            return(1)
        elif Exp_Adv.match(x):
            return(2)
        elif Exp_ID.match(x):
            return(3)
        else:
            return(0)
    except:
        return(0)

def set_email(x):
    try:
        if len(x) > 2:
            return("{0}@cisco.com".format(x))
        else:
            return("Unknown")
    except:
        return("Unknown")
    

def split_email(s1):
    try:
        if len(s1) > 2:
            s2 = s1.split('@')
            return(s2[0])
        else:
            return("Unknown")
    except:
        return("Unknown")

    
def set_dir(x):
    try:
        if len(x) > 2:
            return("http://directory.cisco.com/dir/reports/{0}".format(x))
        else:
            return("Unknown")
    except:
        return("Unknown")


def max_exp_rtr(x):
    y = "Enterprise Routing"
    x1 = INPUT_SAGE1[INPUT_SAGE1['Email1'].str.contains(x, case=False)]
    y1 = x1[x1['Technology Area Name'].str.contains(y, case=False)]
    z = y1['Expertise Value Number'].max()
    if z == 3:
        return("In-Depth")
    elif z == 2:
        return("Advanced")
    elif z == 1:
        return("Foundational")
    else:
        return("Unknown")
    
def max_exp_sw(x):
    y = "Enterprise Switching"
    x1 = INPUT_SAGE1[INPUT_SAGE1['Email1'].str.contains(x, case=False)]
    y1 = x1[x1['Technology Area Name'].str.contains(y, case=False)]
    z = y1['Expertise Value Number'].max()
    if z == 3:
        return("In-Depth")
    elif z == 2:
        return("Advanced")
    elif z == 1:
        return("Foundational")
    else:
        return("Unknown")
    
def max_exp_wrl(x):
    y = "Wireless"
    x1 = INPUT_SAGE1[INPUT_SAGE1['Email1'].str.contains(x, case=False)]
    y1 = x1[x1['Technology Area Name'].str.contains(y, case=False)]
    z = y1['Expertise Value Number'].max()
    if z == 3:
        return("In-Depth")
    elif z == 2:
        return("Advanced")
    elif z == 1:
        return("Foundational")
    else:
        return("Unknown")
    

print("Loading Complete {0}".format(datetime.datetime.now()))    
print("DONE - {0}".format(datetime.datetime.now()))



## Extract Expertise from SAGE

In [ ]:
print("STARTING - {0}".format(datetime.datetime.now()))


# Translate Expertise to Numbers
INPUT_SAGE1['Expertise Value Number'] = INPUT_SAGE1['Expertise Value'].apply(set_expertise)


INPUT_SAGE1['Routing Expertise'] = INPUT_SAGE1['Email1'].apply(max_exp_rtr)
INPUT_SAGE1['Switching Expertise'] = INPUT_SAGE1['Email1'].apply(max_exp_sw)
INPUT_SAGE1['Wireless Expertise'] = INPUT_SAGE1['Email1'].apply(max_exp_wrl)
INPUT_SAGE1['uid'] = INPUT_SAGE1['Email1']

INPUT_SAGE1a = INPUT_SAGE1.loc[:, ['uid', 'Routing Expertise', 'Switching Expertise', 'Wireless Expertise']]
INPUT_SAGE1a.drop_duplicates(inplace=True)

# Determine SEVT Membership
email_key = INPUT_SAGE2.columns.values[0]
INPUT_SAGE1a['EN SEVT Member'] = INPUT_SAGE1a['uid'].isin(INPUT_SAGE2[email_key])

INPUT_DIR1a = INPUT_DIR1.loc[:,('uid', 'firstname', 'lastname', 'title', 'mgr', 'departmentdesc')]
DIRECTORY = INPUT_DIR1a.merge(INPUT_DIR2, on='departmentdesc', how='left')
  
print("DONE - {0}".format(datetime.datetime.now()))



## Custom directory mapping
#### Add any one off modifications here
#### IE:  The TSA's are in COMMERCIAL HQ, but we want to represent them in the Areas they support for reporting purposes

In [ ]:
print("STARTING - {0}".format(datetime.datetime.now()))

def custom_dir(uid, new_value, col_name):
    DIRECTORY[col_name][DIRECTORY['uid'] == uid] = new_value 

custom_dir('anbetz', 'COMMERCIAL EAST AREA', 'L4')
custom_dir('bschnewe', 'COMMERCIAL CENTRAL AREA', 'L4')
custom_dir('chijenki', 'COMMERCIAL SOUTH AREA', 'L4')
custom_dir('dhighlan', 'COMMERCIAL WEST AREA', 'L4')
custom_dir('dsheaffe', 'COMMERCIAL SOUTH AREA', 'L4')
custom_dir('gwerner', 'COMMERCIAL WEST AREA', 'L4')
custom_dir('sclayton', 'COMMERCIAL EAST AREA', 'L4')
custom_dir('tyoutsey', 'COMMERCIAL CENTRAL AREA', 'L4')

print("DONE - {0}".format(datetime.datetime.now()))

## Download Additional SmartSheet DataPoints
#### IE:  SD-WAN POV Certification Status

In [ ]:
# Download fresh data from SmartSheets
print("STARTING - {0}".format(datetime.datetime.now()))

# TODO: Update this with the ID of your sheet to update
sheet_id = 3103471428757380 #SD-WAN POV Tracker
download_path = "."

# Call the download module
response = ss.Sheets.get_sheet_as_excel(sheet_id, download_path)

print("Downloaded '{0}' SmartSheet to '{1}'".format(response.filename,response.download_directory))

downloaded_ss = response.filename

input_file_ss_02 = pd.ExcelFile(downloaded_ss)
cwd = os.getcwd()

INPUT_POVCert_01 = input_file_ss_02.parse(input_file_ss_02.sheet_names[0])
INPUT_POVCert_01['uid'] = INPUT_POVCert_01['Email Address'].apply(split_email)
INPUT_POVCert_01['SD-WAN "DLP" POV Certified'] = INPUT_POVCert_01['Status']
INPUT_POVCert_02 = INPUT_POVCert_01[['uid', 'SD-WAN "DLP" POV Certified']][INPUT_POVCert_01['uid'].isin(DIRECTORY['uid'])]

print("DONE - {0}".format(datetime.datetime.now()))

## Map UserID's to Directory Information

In [ ]:
print("STARTING - {0}".format(datetime.datetime.now()))


SAGE_DIR1 = INPUT_SAGE1a.merge(DIRECTORY, on='uid', how='inner')
SAGE_DIR1 = SAGE_DIR1.merge(INPUT_POVCert_02, on='uid', how='inner')

# Create email and directory links
SAGE_DIR1['Email Address'] = SAGE_DIR1['uid'].apply(set_email)
SAGE_DIR1['Directory'] = SAGE_DIR1['uid'].apply(set_dir)

# Fill missing Region info
SAGE_DIR1 = SAGE_DIR1.fillna('Unknown')
OUTPUT1 = SAGE_DIR1[['L4', 'L5', 'L6', 'uid', 'firstname', 'lastname','title', 'mgr', 'Email Address', 'Directory', 'EN SEVT Member', 'SD-WAN "DLP" POV Certified', 'Routing Expertise','Switching Expertise', 'Wireless Expertise' ]]
OUTPUT1.dropna(axis=0, how='any')
OUTPUT2 = OUTPUT1.drop_duplicates(keep='first')
print("Loaded local CSV files {0}".format(datetime.datetime.now()))
print("DONE - {0}".format(datetime.datetime.now()))



## Write Full Updated Champions Report to Local XLSX

In [ ]:
print("STARTING - {0}".format(datetime.datetime.now()))


writer = pd.ExcelWriter(output_file)
OUTPUT2.to_excel(writer,'EN Champions',index=False)
writer.save()
print("Saved {1} to disk {0}".format(datetime.datetime.now(), 'EN Champions'))
print("DONE - {0}".format(datetime.datetime.now()))



## Export SmartSheets Champion Sheet to local XLSX

In [ ]:
# Download fresh data from SmartSheets
print("STARTING - {0}".format(datetime.datetime.now()))

# TODO: Update this with the ID of your sheet to update
sheet_id = 809447254714244 #Prod EN Champions
# sheet_id = 8459983764383620 #Dev EN Champions
download_path = "."

# Call the download module
response = ss.Sheets.get_sheet_as_excel(sheet_id, download_path)

print("Downloaded '{0}' SmartSheet to '{1}'".format(response.filename,response.download_directory))

downloaded_ss = response.filename
print("DONE - {0}".format(datetime.datetime.now()))



## Determine list of changes from SmartSheet to new Champions report
### Save changes to local XLSX for manual reference

In [ ]:
print("STARTING - {0}".format(datetime.datetime.now()))


input_file04 = pd.ExcelFile(downloaded_ss)
cwd = os.getcwd()

INPUT_OLD = input_file04.parse(input_file04.sheet_names[0])

REMOVE_LIST = INPUT_OLD[~INPUT_OLD['uid'].isin(INPUT_DIR1['uid'])]
ADD_LIST = OUTPUT2[~OUTPUT2['uid'].isin(INPUT_OLD['uid'])]
KEEP_OLD = INPUT_OLD[INPUT_OLD['uid'].isin(INPUT_DIR1['uid'])]
KEEP_NEW = OUTPUT2[OUTPUT2['uid'].isin(INPUT_OLD['uid'])]
KEEP_OLD['EN SEVT Member'].fillna(0, inplace=True)

KEEP_NEW_01 = KEEP_NEW[['L4', 'L5', 'L6', 'uid', 'firstname', 'lastname', 'title', 'mgr', 'Email Address', 'Directory', 'EN SEVT Member', 'SD-WAN "DLP" POV Certified', 'Routing Expertise', 'Switching Expertise', 'Wireless Expertise']].set_index('uid')
KEEP_OLD_01 = KEEP_OLD[['L4', 'L5', 'L6', 'uid', 'firstname', 'lastname', 'title', 'mgr', 'Email Address', 'Directory', 'EN SEVT Member', 'SD-WAN "DLP" POV Certified',  'Routing Expertise', 'Switching Expertise', 'Wireless Expertise']].set_index('uid')

KEEP_NEW_01['EN SEVT Member'] = KEEP_NEW_01['EN SEVT Member'].astype('bool')
KEEP_OLD_01['EN SEVT Member'] = KEEP_OLD_01['EN SEVT Member'].astype('bool')

CHANGE_LIST = pd.DataFrame()
cols = KEEP_OLD_01.columns

update_dict1 = {}

total_records = 0
update_records = 0

for u in KEEP_NEW['uid']:
    try:
        total_records += 1
        if (KEEP_OLD_01.loc[u, :] != KEEP_NEW_01.loc[u, :]).any():
            CHANGE_LIST = CHANGE_LIST.append(KEEP_NEW_01.loc[u, :])
            update_records += 1
            changes = []        
            for col in cols:
                if KEEP_OLD_01.loc[u, col] != KEEP_NEW_01.loc[u, col]:
                    changes.append(col)
                    update_dict1[u] = {col : KEEP_NEW_01.loc[u, col]}
                    #print(u, col, KEEP_NEW_01.loc[u, col])
            CHANGE_LIST.loc[u, "Changes"] = str(changes)
    except:
        print("Could not process {0} - Review Manually".format(u))
        update_dict1[u] = {"Update Needed" : "Error"}

        
print("{0} of {1} records to update.".format(update_records, total_records))
    

CHANGE_LIST['uid'] = CHANGE_LIST.index

output_file = "Changes.xlsx"

writer = pd.ExcelWriter(output_file)
try:
    CHANGE_LIST[['L4', 'L5', 'L6', 'uid', 'firstname', 'lastname', 'title', 'mgr', 'Email Address', 'Directory', 'EN SEVT Member', 'SD-WAN "DLP" POV Certified',  'Routing Expertise', 'Switching Expertise', 'Wireless Expertise', 'Changes']].to_excel(writer,'CHANGE_LIST',index=False)
except:
    print("Error writing CHANGE_LIST to file")
    
try:
    REMOVE_LIST[['L4', 'L5', 'L6', 'uid', 'firstname', 'lastname', 'title', 'mgr', 'Email Address', 'Directory', 'EN SEVT Member', 'SD-WAN "DLP" POV Certified',  'Routing Expertise', 'Switching Expertise', 'Wireless Expertise']].to_excel(writer,'REMOVE_LIST',index=False)
except:
    print("Error writing REMOVE_LIST to file")
    
try:
    ADD_LIST[['L4', 'L5', 'L6', 'uid', 'firstname', 'lastname', 'title', 'mgr', 'Email Address', 'Directory', 'EN SEVT Member', 'SD-WAN "DLP" POV Certified',  'Routing Expertise', 'Switching Expertise', 'Wireless Expertise']].to_excel(writer,'ADD_LIST',index=False)
except:
    print("Error writing ADD_LIST to file")   

writer.save()

print("List of changes saved to {0} - {1}".format(output_file, datetime.datetime.now()))

print("DONE - {0}".format(datetime.datetime.now()))


In [ ]:
print("STARTING - {0}".format(datetime.datetime.now()))

Col_Update = ss.Sheets.get_column_by_title(sheet_id, "Update Needed").id

def update_needed(uid, text):
    r = ss.Sheets.search_sheet(sheet_id, uid)
    r2 = json.loads(str(r))
    Row_Update = r2['results'][0]['objectId']

    # Build new cell value
    new_cell = ss.models.Cell()
    new_cell.column_id = Col_Update
    new_cell.value = str(text)
    new_cell.strict = False

    # Build the row to update
    new_row = ss.models.Row()
    new_row.id = Row_Update
    new_row.cells.append(new_cell)

    # Update rows
    updated_row = ss.Sheets.update_rows(sheet_id, [new_row])
    
for u in REMOVE_LIST['uid']:
    update_needed(u, "REMOVE")
    

column_names = get_column_names(sheet_id)

update_cells_by_dict(update_dict1)
    
print("DONE - {0}".format(datetime.datetime.now()))